### Loading PDF

In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import PyPDF2

# Step 1: Extract text from PDF
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text()
    return text

# Extract text from the PDF
pdf_path = '../data/export.pdf'
text = extract_text_from_pdf(pdf_path)


In [2]:
text

'1/22/24, 4:06 PM Guide to Small Business Exporting\nhttps://www.uschamber.com/co/start/strategy/small-business-exporting-guide 1/12Privacy - Terms\nSTART » STRATEGY\nA Complete Guide to Small Business Exporting\nExporting overseas is challenging but rewarding for small businesses.\nBy: Jessica Elliott , Contributor\nShare-\ue912\ue911\ue910\ue91a\ue909\nThe potential new markets that small businesses can access through exporting can prove worthy of the extra legwork it\nrequires. — ijeab/Getty Images\ue90c\n\ue907\nBy continuing on our website, you agree to our use of cookies for statistical and personalisation purposes.\nKnow More\nI AgreeSAVE YOUR SEAT FOR OUR SMALL BUSINESS UPDATE ON JAN. 31 @ 12PM ET! \ue90d1/22/24, 4:06 PM Guide to Small Business Exporting\nhttps://www.uschamber.com/co/start/strategy/small-business-exporting-guide 2/12Small- and medium-sized enterprises (SMEs) accounted for $460 billion of the known export\nvalue in 2019. Yet, e-commerce skyrocketed in 2020, prov

### Summarize Text

In [3]:
from dotenv import load_dotenv
import os
load_dotenv()
pinecone_api_key = os.getenv("PINECONE_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["PINECONE_API_KEY"] = pinecone_api_key

In [53]:
prompt = """Summarize or shorten the following text and generate possible topics for learning content:\n\n{chunk}\n

Output format:
#starting with open curly bracket
"summary" : #summary in string,
"topics" : #dynamic topics, 3 at most, line by line in a single string
#closing with curly bracket
"""

In [ ]:
# How to use stream
# client = OpenAI()

# stream = client.chat.completions.create(
#     model="gpt-3.5-turbo-1106",
#     messages=[{"role": "user", "content": prompt}],
#     stream=True,
# )
# ans = ''
# for chunk in stream:
#     if chunk.choices[0].delta.content is not None:
#         # print(chunk.choices[0].delta.content, end=" ")
#         ans += chunk.choices[0].delta.content
#         # print(ans)

# return ans

In [58]:
import json
from openai import OpenAI

def summarize_text(text, chunk_size=2000, chunk_overlap=20):
    # Split text into chunks using RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = text_splitter.split_text(text)
    summaries = []
    topics = ''
    
    # Summarize each chunk using OpenAI
    def summarize_topics(chunk):

        client = OpenAI()
        global prompt
        prompt = prompt.format(chunk=chunk)
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        return completion.choices[0].message.content

    # Organize summaries
    for chunk in chunks:
        result = summarize_topics(chunk)
        print("Result:", result)
        print(type(result))
        json_result = json.loads(result)

        summaries.append(json_result['summary'])
        topics += json_result['topics']

    return (summaries, topics)


### Generate Topics

In [93]:
# # Generate Topics from the summary
# def generate_topics(content):
#     client = OpenAI()
#     completion = client.chat.completions.create(
#         model="gpt-4o-mini",
#         messages=[
#             {"role": "user", "content": prompt}
#         ]
#     )
#     return completion.choices[0].message.content

In [59]:
sums, topics = summarize_text(text=text)

Result: {
"summary" : "Exporting can be both challenging and rewarding for small businesses, as it enables access to new international markets, potentially enhancing growth and resilience. While exporting offers numerous benefits, such as increased profits and reduced dependency on the domestic economy, it also carries risks, including regulations and logistics delays. Small- and medium-sized enterprises (SMEs) accounted for significant export value in 2019, and the rise of e-commerce provides new opportunities for global sales. Business owners must carefully research and plan their export strategies.",
"topics" : "Benefits of exporting for small businesses\nRisks and challenges of international trade\nChoosing goods and services for export"
}
<class 'str'>
Result: {
"summary" : "Exporting presents both challenges and rewards for small businesses. With SMEs contributing $460 billion to U.S. exports in 2019, international sales offer potential growth opportunities, especially after the 

In [60]:
sums

['Exporting can be both challenging and rewarding for small businesses, as it enables access to new international markets, potentially enhancing growth and resilience. While exporting offers numerous benefits, such as increased profits and reduced dependency on the domestic economy, it also carries risks, including regulations and logistics delays. Small- and medium-sized enterprises (SMEs) accounted for significant export value in 2019, and the rise of e-commerce provides new opportunities for global sales. Business owners must carefully research and plan their export strategies.',
 'Exporting presents both challenges and rewards for small businesses. With SMEs contributing $460 billion to U.S. exports in 2019, international sales offer potential growth opportunities, especially after the surge of e-commerce in 2020. Benefits of exporting include market diversification, increased resilience to economic downturns, and enhanced profit margins. However, businesses must navigate risks suc

In [61]:
print(topics)

Benefits of exporting for small businesses
Risks and challenges of international trade
Choosing goods and services for exportBenefits of Exporting for Small Businesses
Risks Associated with International Trade
Strategies for Selecting Goods to ExportBenefits of Exporting for Small Businesses
Risks and Challenges of International Trade
Strategies for Successful ExportingBenefits of Exporting for Small Businesses
Risks and Challenges in International Exporting
Strategies for Effective Market EntryBenefits of Exporting for SMEs
Risks and Challenges in Exporting
Choosing Goods or Services for ExportBenefits of Exporting for Small Businesses
Challenges and Risks in International Trade
Strategies for Successful ExportingBenefits of Exporting for Small Businesses
Risks and Challenges in International Trade
Selecting Goods and Services for ExportBenefits of Exporting for Small Businesses
Risks and Challenges in International Trade
Choosing Goods and Services for Export
